In [19]:

import os
from dotenv import load_dotenv
import requests
import json

# cargamos el archivo .env que tiene nuestras credenciales
load_dotenv()
CID = os.getenv("CLIENT_ID")
CSEC = os.getenv("CLIENT_SECRET")

# muestro solo un pedacito de las credenciales para chequear que se cargaron
print("CLIENT_ID:", CID[:10], "...")
print("CLIENT_SECRET:", CSEC[:10], "...")


# Pedimos el token a Spotify
url_token = "https://accounts.spotify.com/api/token"
data = {"grant_type": "client_credentials"}

resp = requests.post(url_token, data=data, auth=(CID, CSEC))
# pedimos que nos den el token. Si falla va a tirar error feo jaja
resp.raise_for_status()
token = resp.json().get("access_token")
print("Token generado:", token[:30], "...")



# Buscar el artista que queremos (se puede cambiar el nombre)
headers = {"Authorization": f"Bearer {token}"}
nombre_artista = "La Vela Puerca"   # podés cambiarlo por tu favorito

url_busqueda = "https://api.spotify.com/v1/search"
params = {"q": nombre_artista, "type": "artist", "limit": 1}

r = requests.get(url_busqueda, headers=headers, params=params)
r.raise_for_status()

data = r.json()
items = data.get("artists", {}).get("items", [])

if not items:
    print("No encontré artista con ese nombre")
    artista_id = None
else:
    artista_id = items[0]["id"]
    artista_nombre_real = items[0]["name"]

    print("Artista encontrado:", artista_nombre_real)
    print(" Artist ID:", artista_id[:10])


# Si encontramos artista, buscamos el top 10
if artista_id:
    url_top = f"https://api.spotify.com/v1/artists/{artista_id}/top-tracks"
    params = {"market": "UY"}

    rtop = requests.get(url_top, headers=headers, params=params)
    rtop.raise_for_status()

    tracks = rtop.json().get("tracks", [])[:10]
    print("Cantidad de temas encontrados:", len(tracks))

    resultado = []
    for t in tracks:
        # armo el diccionario con los datos que pide el trabajo
        resultado.append({
            "song": t["name"],
            "popularity": t["popularity"],
            "duration": t["duration_ms"]  # viene en milisegundos
        })

    # guardo el resultado en un archivo JSON
    with open("top10.json", "w", encoding="utf-8") as f:
        json.dump(resultado, f, ensure_ascii=False, indent=2)

    # muestro cada tema en formato más lindo
    for i, t in enumerate(resultado, start=1):
        segundos = t["duration"] // 1000
        mins, secs = divmod(segundos, 60)
        print(f"{i:02d}. {t['song']} | pop: {t['popularity']} | dur: {mins}:{secs:02d}")



CLIENT_ID: 5192d08007 ...
CLIENT_SECRET: 13464fb5b4 ...
Token generado: BQBT6y-YOnFdFWknOeAJ9-xLQ8Eg87 ...
Artista encontrado: La Vela Puerca
 Artist ID: 6nVcjUJemq
Cantidad de temas encontrados: 10
01. Zafar | pop: 44 | dur: 4:22
02. Va a Escampar | pop: 46 | dur: 4:41
03. El Viejo | pop: 43 | dur: 3:07
04. Llenos de Magia | pop: 45 | dur: 3:52
05. Para No Verme Más | pop: 59 | dur: 4:05
06. Clarobscuro | pop: 39 | dur: 5:08
07. En el Limbo | pop: 36 | dur: 4:42
08. De Atar | pop: 42 | dur: 4:04
09. Por la Ciudad | pop: 39 | dur: 3:02
10. Ves | pop: 50 | dur: 3:15
